In [31]:
from datasets import load_dataset
import pandas as pd
from dotenv import load_dotenv
import openai
import os

load_dotenv()

True

In [32]:
train = pd.read_csv("../data/corpus_train.csv")

In [43]:
few_shots = train.sample(20)

In [44]:
few_shots["label"].value_counts("label")

assumption       0.50
anecdote         0.20
testimony        0.15
statistics       0.05
title            0.05
common-ground    0.05
Name: label, dtype: float64

In [45]:
all_data = load_dataset("csv", data_files={'train': ["../data/corpus_train.csv"], 'validation': ["../data/corpus_valid.csv"], "test": ["../data/corpus_test.csv"]})

Using custom data configuration default-2c05e3db047a88f3
Reusing dataset csv (/home/georg/.cache/huggingface/datasets/csv/default-2c05e3db047a88f3/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/3 [00:00<?, ?it/s]

In [46]:
def apply_promt(inp):
    prompt = ""

    for _,row in few_shots.iterrows():
        prompt += "prompt: " + row["sentence"] + "\nlabel: " + row["label"] + "<end>\n"

    prompt += "prompt: " + inp["sentence"] + "\nlabel:"

    return {"prompt": prompt}


prompted_validation = all_data["validation"].map(apply_promt)

  0%|          | 0/2716 [00:00<?, ?ex/s]

In [49]:
import time

key = os.getenv("OPENAI_API_KEY")
count = 50

answers = []

for i, prompt in enumerate(prompted_validation[:50]["prompt"]):

  response = openai.Completion.create(
    engine="text-davinci-001",
    prompt=prompt,
    temperature=0,
    max_tokens=6,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["<end>"]
  )

  print(f"{i}/{len(prompted_validation)}")

  time.sleep(0.5)

  answers.append(response.choices[0].text)


answers = [x.strip() for x in answers] 

0/2716
1/2716
2/2716
3/2716
4/2716
5/2716
6/2716
7/2716
8/2716
9/2716
10/2716
11/2716
12/2716
13/2716
14/2716


RateLimitError: Rate limit reached for default. Limit: 20.000000 / min. Current: 24.000000 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://beta.openai.com/account/billing to add a payment method.

In [56]:
from sklearn.metrics import classification_report

print(classification_report(prompted_validation["label"][:15], answers))

              precision    recall  f1-score   support

      advice       0.00      0.00      0.00       0.0
    anecdote       0.00      0.00      0.00       0.0
  assumption       0.00      0.00      0.00       0.0
  statistics       0.00      0.00      0.00       0.0
   testimony       0.00      0.00      0.00       0.0
       title       0.00      0.00      0.00       0.0
    anecdote       0.00      0.00      0.00       4.0
  assumption       0.00      0.00      0.00       6.0
  statistics       0.00      0.00      0.00       1.0
   testimony       0.00      0.00      0.00       4.0

    accuracy                           0.00      15.0
   macro avg       0.00      0.00      0.00      15.0
weighted avg       0.00      0.00      0.00      15.0



/home/georg/.cache/pypoetry/virtualenvs/ltp-project-JPCASTvD-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/georg/.cache/pypoetry/virtualenvs/ltp-project-JPCASTvD-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/georg/.cache/pypoetry/virtualenvs/ltp-project-JPCASTvD-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division